# Twilio SMS Getting Started Guide

## Finding and Purchasing an Available Number and Sending a Message

This guide will walk you through step-by-step getting started with Twilio SMS

# Walkthrough
First, we need to import os and Client from twilio.rest

In [411]:
import os
from twilio.rest import Client

Next, you will want to create another file, info.py, in which you will store your account SID and auth token. Name the variables accountSID and authToken respectively. Then, import the file so that we can access these variables.

In [412]:
import info

Then we will create a client using these variables:

In [413]:
def create_client():
    try:
        account_sid = os.environ.get("ACCOUNT_SID", info.accountSID)
        auth_token = os.environ.get("AUTH_TOKEN", info.authToken)
        client = Client(account_sid, auth_token)
        return client
    except AttributeError:
        print("Make sure you correctly type your personal information in a file named 'info.py'.")
        return None

Using our client, we want to create a function that will search for available phone numbers so we may purchase one to use. I have chosen to search for available phone numbers by area code in the US with the area code '540'. You can change this to whatever area code you prefer.

This function with create a list **local** of 20 available phone numbers. 

After creating the list of local numbers, the function will pick the first one from the list, change its format to '+1xxxxxxxxxx', and return the number.

In [414]:
def get_number(client, area_code):
    try:
        limit = 20
        local = client.available_phone_numbers('US').local.list(
                                                                area_code,
                                                                limit
                                                            )
        number = local[0].friendly_name
        my_number = "+1" + number[1:4] + number[6:9] + number[10:14]
        return my_number
    except twilio.base.exceptions.TwilioException:
        return ("Your Twilio credentials were incorrect.")
    except IndexError:
        return("Area code entered was invalid.")

Now that I have my number, the following **purchase_number** function will purchase it on my account:

In [415]:
def purchase_number(client, my_number):
    try:
        incoming_phone_number = client.incoming_phone_numbers \
                                        .create(phone_number=my_number)
        return("Number has successfully been purchased.")
    except twilio.base.exceptions.TwilioRestException:
        return("Problem with twilio account.")

Finally, I can send my message with the function **send_message** below. This will send my message from the number I have just purchased to my personal number stored as a variable in my info.py file. You can change the body to say whatever you want. 

In [416]:
def send_message(client, my_number, personalNum):
    message = client.messages \
                    .create(
                         body="This is my message.",
                         from_= my_number,
                         to= personalNum
                     )

    return message.sid

After creating our functions we can run them. First we will create our client variable.

In [417]:
client = create_client()

Next we'll use our **get_number** function to find a local number for us to purchase and print it. Before running, the console will ask us to input and area code to search for.

In [418]:
area_code = input("Type area code to search for: ")
my_number = get_number(client, area_code)
print("This is my Twilio number: " + my_number)

Type area code to search for: 540
This is my Twilio number: +15402022093


The next cell will purchase the number we just found.

In [419]:
print(purchase_number(client, my_number))

Problem with twilio account.


The last cell will send a message from the purchased number to you personal number.

In [425]:
message_sid = send_message(client, my_number, info.personalNum)

TwilioRestException: HTTP 400 error: Unable to create record: The From phone number +15402022093 is not a valid, SMS-capable inbound phone number or short code for your account.

# Testing Our Code

In [420]:
import unittest
import re

In [421]:
class TestNotebook(unittest.TestCase):
    def test_info_py_format(self):
        #Is info.personalNum in e. 164 format?
        self.assertTrue(re.match('^\+[1-9]\d{1,14}$', info.personalNum))
    def test_get_number_output_format(self):
        account_sid = os.environ.get("ACCOUNT_SID", info.accountSID)
        auth_token = os.environ.get("AUTH_TOKEN", info.authToken)
        client = Client(account_sid, auth_token)
        #Is get_number() output in e. 164 format?
        self.assertTrue(re.match('^\+[1-9]\d{1,14}$', get_number(client, '540')))
    def test_wrong_credentials(self):
        account_sid = os.environ.get("ACCOUNT_SID", '1')
        auth_token = os.environ.get("AUTH_TOKEN", '2')
        client = Client(account_sid, auth_token)
        self.assertEqual(get_number(client, '540'), "Your Twilio credentials were incorrect.")
    def test_wrong_area_code(self):
        self.assertEqual(get_number(client, '1'), "Area code entered was invalid.")

In [422]:
unittest.main(argv=[''], verbosity=2, exit=False)

test_get_number_output_format (__main__.TestNotebook) ... ok
test_info_py_format (__main__.TestNotebook) ... ok
test_wrong_area_code (__main__.TestNotebook) ... ok
test_wrong_credentials (__main__.TestNotebook) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.732s

OK
